In [ ]:
import time
import random
from random import randint
import math as math

In [ ]:
    node = []
    sendList = []
    recieveList = []
    nextList = []

    def rangeTest(x, y, rangeRadius, x1, y1):
      distance = math.sqrt(((x - x1) ** 2) + ((y - y1) ** 2))
      if(distance <= rangeRadius):
        return True
      else:
        return False

In [ ]:
class Header:

    # Construtor
    def __init__(self, type, addrOrigin, addrDest, num, req, seqNum, seqList):

        # se pertencer à camada de Enlace:
        if(type == "ENLACE"):
            self._type = "ENLACE"
            self._addrOrigin = addrOrigin
            self._addrDest = addrDest
            self._num = num

        # se pertencer à camada de Rede:
        elif(type == "REDE"):
            self._type = "REDE"
            self._addrDest = addrDest
            self._req = req
            self._seqNum = seqNum
            self._seqList = seqList


In [ ]:
class Package:
    def __init__(self, message):

        self._id = -1
        self._headers = []
        self._data = message

    # metodos de pacote

    def createPackage(message):
        return Package(message)

    # metodos do header

    def addHeader(self, header):
        self._headers.append(header)

    def getHeaderEnlace(self):
        for header in self._headers:
            if(header._type == "ENLACE"):
                return header

    def getHeaderRede(self):
        for header in self._headers:
            if(header._type == "REDE"):
                return header

    def updateSequence(self, sequence):
        for header in self._headers:
            if(header._type == "REDE"):
                header._seqList = sequence


In [ ]:
class Route:

    def __init__(self, dest, sequence):

        self._dest = dest
        self._sequence = sequence

In [ ]:
class CamadaFisica:

    def __init__(self, x, y, id, rangeRadius):
        self._x = x
        self._y = y
        self._id = id
        self._rangeRadius = rangeRadius

        self._neighbor = []
        self._sendPackages = []
        self._recievePackages = []
        self._packageBackup = []

    def findNeighbor(self):
        for no in node:
            if((no._id != self._id) and (rangeTest(self._x, self._y, self._rangeRadius, no._camadaRede._camadaEnlace._camadaFisica._x, no._camadaRede._camadaEnlace._camadaFisica._y))):
                if(no not in self._neighbor):
                    self._neighbor.append(no)

    def recievePackage(self, package):
        recieveList.append(self._id)
        self._recievePackages.append(package)

    def sendPackage(self):
        self.findNeighbor()
        for no in self._neighbor:
            no._camadaRede._camadaEnlace._camadaFisica.recievePackage(
                self._sendPackages[0])
        self._packageBackup.append(self._sendPackages.pop(0))

In [ ]:
class CamadaEnlace:

    def __init__(self, _camadaFisica):

        self._backoff = 0
        self._meioLivre = True
        self._camadaFisica = _camadaFisica
        self._packagesLidos = []

    def acessoMeio(self):
        if(self._camadaFisica._recievePackages == []):
            return True
        else:
            return False

    def recievePackage(self):
        if(len(self._camadaFisica._recievePackages) > 1):
            self._camadaFisica._recievePackages = []
            print("Node ", self._camadaFisica._id, ": Collision")

        elif(len(self._camadaFisica._recievePackages) == 1):
            pacote = self._camadaFisica._recievePackages.pop(0)
            header = pacote.getHeaderEnlace()
                
            if(header._addrDest == self._camadaFisica._id):
                self._packagesLidos.append(pacote)

            elif(header._addrDest  == -1):
                self._packagesLidos.append(pacote)

    def addPackage(self, pacote, endDestino):
        header = Header("ENLACE", self._camadaFisica._id, endDestino, 0, -1, -1, -1)
        pacote.addHeader(header)
        self._camadaFisica._sendPackages.append(pacote)

    def enviarPacote(self):
        
        self._meioLivre = self.acessoMeio()
        if(self._meioLivre == True):
            
            #Verifica se nó não está em backoff 
            if(self._camadaFisica._sendPackages != []):
                if(self._backoff == 0):
                    self._camadaFisica.sendPackage()
                 
                else:
                    nextList.append(self._camadaFisica._id)
                    self._backoff -= 1

        #Caso o meio estiver ocupado o nó entrará em backoff
        elif(self._camadaFisica._sendPackages != []):
            
            if(self._backoff == 0):
                self._backoff = randint(2, 5)
                nextList.append(self._camadaFisica._id)



In [ ]:
class CamadaRede:
  def __init__(self, camadaEnlace): 
        
        self._camadaEnlace = camadaEnlace
        self._listaPacotes = []
        self._listRREQs = []
        self._waitingRotaToList = []
        self._rotas = []

  def adicionarPacote(self, endDestino, mensagem):
        pacote = Package(mensagem)
        cabecalho = Header("REDE",self._camadaEnlace._camadaFisica._id, endDestino, -1, -1, -1, None)
        pacote.addHeader(cabecalho)
        self._listaPacotes.append(pacote)
  
  def enviarRREP(self, endDestino, sequencia, rota):
        
        #Cria um pacote e insere o cabeçalho da camada de rede
        cabecalho = Header("REDE", self._camadaEnlace._camadaFisica._id, endDestino, -1, 1, -1, sequencia)
        pacote = Package(rota)
        pacote.addHeader(cabecalho)
        
        print("Node ", self._camadaEnlace._camadaFisica._id, ": Sending RREP to node ", cabecalho._addrDest)
        
        #Define a rota requisitada
        for index,mac in enumerate(cabecalho._seqList):
            if(mac == self._camadaEnlace._camadaFisica._id):       
                proximoDestino = cabecalho._seqList[index+1]
                proximoPacote = pacote
                self._camadaEnlace.addPackage(proximoPacote, proximoDestino)
                break
       
  def enviarRREQ(self, endDestino):
        
        #Inicializa uma sequencia e coloca o seu ID como primeiro
        sequencia = []
        sequencia.append(self._camadaEnlace._camadaFisica._id)
        seqNumero = random.randint(1,128733788) 
        self._listRREQs.append(seqNumero)

        #Cria um pacote e insere o cabeçalho da camada de rede
        cabecalho = Header("REDE", self._camadaEnlace._camadaFisica._id, endDestino, -1, 0, seqNumero, sequencia)
        pacote = Package("")
        pacote.addHeader(cabecalho)
        
        print("Node ", self._camadaEnlace._camadaFisica._id, ": Sending RREQ to node ", cabecalho._addrDest)
        self._camadaEnlace.addPackage(pacote, -1)


  def receberPacote(self):

        #Chama a função de tratar pacote recebido na camada de enlace
        self._camadaEnlace.recievePackage()
          
        #Verifica se tem pacotes recebidos
        if(self._camadaEnlace._packagesLidos != []):

            pacote = self._camadaEnlace._packagesLidos.pop(0)
            cabecalho = pacote.getHeaderRede()

            #Se o pacote for recebido for de dados
            if(cabecalho._req == -1):
                
                #Verifica se o pacote é para mim
                if(cabecalho._addrDest == self._camadaEnlace._camadaFisica._id):
                    print("Node ", self._camadaEnlace._camadaFisica._id, ": Package received: ", pacote._data)
                else:
                    print("Node ", self._camadaEnlace._camadaFisica._id, ": The package isn't for me")
                    print("Node ", self._camadaEnlace._camadaFisica._id, ": Sending package to the next node")
                    for index,mac in enumerate(pacote._headers[0]._seqList):
                        if(mac == self._camadaEnlace._camadaFisica._id):
                            proximoDestino = cabecalho._seqList[index-1]
                            break

                    pacote._headers.pop(1)
                    self._camadaEnlace.addPackage(pacote, proximoDestino)
                    sendList.append(self._camadaEnlace._camadaFisica._id)


            #Se o pacote for recebido for um RREQ
            elif(cabecalho._req == 0):
                
                print("Node ", self._camadaEnlace._camadaFisica._id, ": RREQ arrived ", cabecalho._seqNum )
                
                #Verifica se eu já recebi esse RREQ
                if(not cabecalho._seqNum in self._listRREQs):
                    self._listRREQs.append(cabecalho._seqNum)
                    cabecalho._seqList.append(self._camadaEnlace._camadaFisica._id)

                    #Verifica se o RREQ é para mim
                    if(cabecalho._addrDest == self._camadaEnlace._camadaFisica._id):
                        print("Node ", self._camadaEnlace._camadaFisica._id, ": I'm the RREQ destination")
                        rota = cabecalho._seqList
                        endDestino = rota[0]
                        sequenciaAteFonte = rota 
                        sequenciaAteFonte.reverse()
                        self.enviarRREP(endDestino,sequenciaAteFonte, rota)
                        sendList.append(self._camadaEnlace._camadaFisica._id)

                    else:
                        print("Node ", self._camadaEnlace._camadaFisica._id, ": RREQ isn't for me")
                        self._camadaEnlace.addPackage(pacote, -1)
                        sendList.append(self._camadaEnlace._camadaFisica._id)
                
                else:
                    print("Node ", self._camadaEnlace._camadaFisica._id, ": RREQ already in list ", cabecalho._seqNum)


            #Se o pacote for recebido for um RREP
            elif(cabecalho._req == 1):
                
                destino = cabecalho._addrDest
                print("Node ", self._camadaEnlace._camadaFisica._id, ": RREP arrived: ", cabecalho._seqList)
                
                #Verifica se o RREP é para mim
                if(destino == self._camadaEnlace._camadaFisica._id):
                    print("Node ", self._camadaEnlace._camadaFisica._id, ": I'm the RREP destination")
                    print("Node ", self._camadaEnlace._camadaFisica._id, ": Sending data package")
                    sequenciaAteDestino = pacote._data
                    rota = Route(cabecalho._seqList[0],sequenciaAteDestino)
                    self._rotas.append(rota)
                    sendList.append(self._camadaEnlace._camadaFisica._id)

                else:
                    
                    print("Node ", self._camadaEnlace._camadaFisica._id, ": RREP isn't for me")
                    
                    for index,mac in enumerate(cabecalho._seqList):
                        if(mac == self._camadaEnlace._camadaFisica._id):
                            proximoDestino = cabecalho._seqList[index+1]
                            proximoPacote = pacote
                            pacote._headers.pop(1)
                            self._camadaEnlace.addPackage(proximoPacote, proximoDestino)
                            sendList.append(self._camadaEnlace._camadaFisica._id)
                            break


  def enviarPacote(self):
        
        #Verifica pacotes a serem enviados
        if(self._listaPacotes != []):

            pacote = self._listaPacotes[0]
            cabecalho = pacote.getHeaderRede()
            sequencia = None

            for rota in self._rotas:
                if(rota._dest == pacote._headers[0]._addrDest):
                    sequencia = rota._sequence  
                    if (pacote._headers[0]._addrDest in self._waitingRotaToList):
                        self._waitingRotaToList.remove(pacote._headers[0]._addrDest)


            #Verifica se a rota para o destino é conhecida
            if(sequencia != None):
                pacote.updateSequence(sequencia)
                self._listaPacotes.pop(0)

                for index,mac in enumerate(pacote._headers[0]._seqList):
                    if(mac == self._camadaEnlace._camadaFisica._id):
                        proximoDestino = cabecalho._seqList[index-1]
                        break
                             
                self._camadaEnlace.addPackage(pacote, proximoDestino)
                sendList.append(self._camadaEnlace._camadaFisica._id)

            elif(not cabecalho._addrDest in self._waitingRotaToList):

                self._waitingRotaToList.append(pacote._headers[0]._addrDest)
                self.enviarRREQ(pacote._headers[0]._addrDest)
        

        #Chama a função de enviar pacotes da camada de enlace
        self._camadaEnlace.enviarPacote()


In [ ]:
class Node:

    #Construtor da classe nó
    def __init__(self, id, x, y, alcance):
        
        #Identificador único para o nó
        self._id = id

        #Adiciona a própria instacia na lista de Nós globais
        node.append(self)

        #Inicializa a camada de rede
        self._camadaRede = CamadaRede(CamadaEnlace(CamadaFisica(x, y, id, alcance)))


    #Função para criar um pacote
    def criarPacote(self, endDestino, mensagem):
        
        print("Node ", self._camadaRede._camadaEnlace._camadaFisica._id, ": Package created to node ", endDestino )
        self._camadaRede.adicionarPacote(endDestino, mensagem)

In [ ]:
node = []
a = Node(0,1,2,1.5)
b = Node(1,2,2,1.5)
c = Node(2,3,2,1.5)
#d = Node(3,4,2,1.5)
#e = Node(4,5,2,1.5)
#f = Node(5,6,2,1.5)

y = True

#Loop do tempo
for i in range(100000):

    #Loop para percorrer 
    for no in node:
        
        #Numero aleatorio entre 0 e 100 para probabilidade de criação de pacotes
        rand = random.randint(0, 100)

        #Numero aleatorio entre 0 e a quantidade de nós, para escolher um para enviar
        envio = random.randint(0, len(node)-1)
        #Teste se rand é menor que 15, gerando assim uma probabilidade de 0.15
        if(rand < 15):
            
                #Se o ID do destino for diferente do dele mesmo, adiciona o pacote no Nó
                if(envio != no._camadaRede._camadaEnlace._camadaFisica._id and y == True):
                        y = False
                        no.criarPacote(envio, "TEST PACKAGE")
                        sendList.append(no._camadaRede._camadaEnlace._camadaFisica._id)
          
    if(nextList != []):
        for i in nextList:
                sendList.append(i)
    del nextList[:]

    #Existe algum nó querendo receber, recebe
    for j in recieveList:
            
            node[j]._camadaRede.receberPacote()
    del recieveList[:]
    
    #Existe algum nó querendo transmitir, transmite naquele instante se posssível
    for i in sendList: 
            node[i]._camadaRede.enviarPacote()
    del sendList[:]


Node  0 : Package created to node  1
Node  0 : Sending RREQ to node  1
Node  1 : RREQ arrived  13571608
Node  1 : I'm the RREQ destination
Node  1 : Sending RREP to node  0
Node  0 : RREP arrived:  [1, 0]
Node  0 : I'm the RREP destination
Node  0 : Sending data package
Node  1 : Package received:  TEST PACKAGE
